# Creating a Real-Time Inferencing Service
After training a predictive model, you can deploy it as a real-time service that clients can use to get predictions from new data.

Creating a Real-Time Inferencing Service
After training a predictive model, you can deploy it as a real-time service that clients can use to get predictions from new data.

Before You Start
Before you start this lab, ensure that you have completed the Create an Azure Machine Learning Workspace and Create a Compute Instance tasks in Lab 1: Getting Started with Azure Machine Learning. Then open this notebook in Jupyter on your Compute Instance.

Connect to Your Workspace
The first thing you need to do is to connect to your workspace using the Azure ML SDK.

In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

UserErrorException: UserErrorException:
	Message: We could not find config.json in: C:\Users\Andy Oduro\Documents\GitHub\festman-diabetes or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "We could not find config.json in: C:\\Users\\Andy Oduro\\Documents\\GitHub\\festman-diabetes or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories."
    }
}

In [ ]:

from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-training")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = LogisticRegression().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# Complete the run
run.complete()

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')